In [374]:
import pandas as pd
import psycopg2
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import re

url = "https://caioicy.github.io/slsa/leaderboards/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"}
req = Request(url, headers=headers)
response = urlopen(req)
html = response.read()
soup = BeautifulSoup(html, 'html.parser')

html_string = str(html)

html_string_sem_colchetes = html_string.replace('[', '!')
html_string_treated = html_string_sem_colchetes.replace(']', '!')
display(html_string_treated)


country_codes = re.findall(r'countryCode\\\\":(.*?),', html_string_treated)
slippi_connect_codes = re.findall(r'slippiConnectCodes\\\\":!(.*?),', html_string_treated)
del country_codes[51] # foi identificado 1 jogador que possui país associado, mas não possui código. aqui, eu apago
country_codes = [code.replace('\\', '') for code in country_codes]
country_codes = [code.replace('"', '') for code in country_codes]
country_codes = [code.replace('}', '') for code in country_codes]
slippi_connect_codes = [code.replace('\\', '') for code in slippi_connect_codes]
slippi_connect_codes = [code.replace('!', '') for code in slippi_connect_codes]
slippi_connect_codes = [code.replace('}', '') for code in slippi_connect_codes]
slippi_connect_codes = [code.replace('"', '') for code in slippi_connect_codes]


# codigo para encontrar o texto que varia = (.*?)
# como estão os dados lá no html:
# \\\\"tr\\\\":{\\\\"slug\\\\":\\\\"tr\\\\",\\\\"tag\\\\":\\\\"TXR\\\\",
# \\\\"countryCode\\\\":\\\\"br\\\\",\\\\"slippiConnectCodes\\\\":c\\\\"TXR#205\\\\"u,\\\\"subregion\\\\":\\\\"br\\\\"}

countrysize = len(country_codes)
codesize = len(slippi_connect_codes)

print("paises encontrados , codes encontrados")
print( countrysize, codesize)

csvcodes = pd.DataFrame({'codes': slippi_connect_codes})
csvpaises = pd.DataFrame({'paises': country_codes})
csvcodes.to_csv('codigos_brutos.csv', index=False)
csvpaises.to_csv('paises_brutos.csv', index=False)


df = pd.DataFrame({'CountryCode': country_codes, 'SlippiConnectCodes': slippi_connect_codes})
display(df)

conexao = psycopg2.connect(database="melee",
                           host="localhost",
                           user="postgres",
                           password="postgres",
                           port="5432")

cursor = conexao.cursor()
cursor.execute("SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'melee_paises')")
table_exists = cursor.fetchone()[0]

if table_exists:
    for index, row in df.iterrows():
        cursor.execute("DROP TABLE melee_paises")
        cursor.execute('''
        CREATE TABLE melee_paises (
            ID SERIAL PRIMARY KEY,
            COUNTRYCODE VARCHAR(255),
            SLIPPICONNECTCODES VARCHAR(255)
        )
    ''')
    for index, row in df.iterrows():
        cursor.execute("INSERT INTO melee_paises (COUNTRYCODE, SLIPPICONNECTCODES) VALUES (%s, %s)",
                       (row['CountryCode'], row['SlippiConnectCodes']))

        
else:
    cursor.execute('''
        CREATE TABLE melee_paises (
            ID SERIAL PRIMARY KEY,
            COUNTRYCODE VARCHAR(255),
            SLIPPICONNECTCODES VARCHAR(255)
        );
    ''')

    for index, row in df.iterrows():
        cursor.execute("INSERT INTO melee_paises (COUNTRYCODE, SLIPPICONNECTCODES) VALUES (%s, %s)",
                       (row['CountryCode'], row['SlippiConnectCodes']))

conexao.commit()
conexao.close()


'b\'<!doctype html>\\n<html lang="en">\\n  <head>\\n    <meta charset="utf-8" />\\n    <link rel="icon" href="/slsa/favicon.png" />\\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\\n    \\n\\t\\t<link href="/slsa/_app/immutable/assets/0.BMS8GDwi.css" rel="stylesheet">\\n\\t\\t<link href="/slsa/_app/immutable/assets/3.CfFntViG.css" rel="stylesheet">\\n\\t\\t<link href="/slsa/_app/immutable/assets/characterStock.BVpLNII1.css" rel="stylesheet">\\n\\t\\t<link rel="modulepreload" href="/slsa/_app/immutable/entry/start.HHSCREsW.js">\\n\\t\\t<link rel="modulepreload" href="/slsa/_app/immutable/chunks/entry.C1r2K6n4.js">\\n\\t\\t<link rel="modulepreload" href="/slsa/_app/immutable/chunks/scheduler.BBINyCwK.js">\\n\\t\\t<link rel="modulepreload" href="/slsa/_app/immutable/chunks/paths.TgIBNfva.js">\\n\\t\\t<link rel="modulepreload" href="/slsa/_app/immutable/entry/app.DIPO3G_P.js">\\n\\t\\t<link rel="modulepreload" href="/slsa/_app/immutable/chunks/index.BWnIrRdG.js

paises encontrados , codes encontrados
289 289


,CountryCode,SlippiConnectCodes
0,ve,9#11
1,cl,MIGUEL#0
2,cl,ILDE#538
3,br,E#9
4,cl,ANON#329
...,...,...
284,pe,BLST#849
285,br,LUPI#915
286,pe,NEON#519
287,cl,COJI#593
